In [4]:
!pip install streamlit

  Using cached protobuf-3.20.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp39-cp39-macosx_10_9_x86_64.whl (982 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grakn-client 2.0.1 requires grpcio==1.36.1, but you have grpcio 1.62.0 which is incompatible.
grakn-client 2.0.1 requires protobuf==3.15.5, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.62.0 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.
typedb-client 2.9.0 requires grpcio==1.43.0, but you have grpcio 1.62.0 which is incompatible.

In [13]:
!pip list

Package                                 Version              Editable project location
--------------------------------------- -------------------- ---------------------------------------------------------
absl-py                                 1.4.0
accelerate                              0.20.3
aiohttp                                 3.9.3
aiosignal                               1.3.1
alabaster                               0.7.12
alembic                                 1.11.1
altair                                  4.2.2
anaconda-client                         1.11.0
anaconda-navigator                      2.2.0
anaconda-project                        0.11.1
ansi2html                               1.8.0
anyio                                   3.7.1
apache-airflow                          2.6.2
apache-airflow-providers-amazon         8.3.0
apache-airflow-providers-apache-kafka   1.1.2
apache-airflow-providers-common-sql     1.5.1
apache-airflow-providers-ftp            3.4.1
apache-

prison                                  0.2.1
prometheus-client                       0.14.1
prompt-toolkit                          3.0.36
proto-plus                              1.23.0
protobuf                                3.20.3
psutil                                  5.9.5
ptyprocess                              0.7.0
pure-eval                               0.2.2
py                                      1.11.0
py4j                                    0.10.9.5
pyarrow                                 10.0.1
pyasn1                                  0.5.0
pyasn1-modules                          0.3.0
pycodestyle                             2.7.0
pycosat                                 0.6.4
pycparser                               2.21
pyct                                    0.4.8
pycurl                                  7.45.1
pydantic                                1.10.9
pydeck                                  0.8.0
pydocstyle                              6.3.0
pyee                    

In [14]:
!python --version

Python 3.9.7


In [6]:
!streamlit --version

2024-04-18 09:03:26.680 INFO    numexpr.utils: NumExpr defaulting to 8 threads.
Streamlit, version 1.12.0


In [1]:
!streamlit hello

2024-04-18 08:35:24.787 INFO    numexpr.utils: NumExpr defaulting to 8 threads.

  Welcome to Streamlit. Check out our demo in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.19:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


^C
  Stopping...


In [7]:
%%writefile try.py
import streamlit as st
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import ChatVertexAI, HarmBlockThreshold, HarmCategory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.runnables import RunnablePassthrough


import time

def stream_data(text):
    for word in text.split():
        yield word + " "
        time.sleep(0.04)

#authenticate project_id
from google.cloud import aiplatform
import vertexai
PROJECT_ID="scholastic-genai-415115"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

def initialize_llm():
    llm = ChatVertexAI(
    model_name="gemini-pro",
    project_id=PROJECT_ID,
    location=REGION,
     safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
   },)
    return llm
st.title("🦜🔗Ask About Langchain")

#st.set_page_config(page_title="🦜🔗Chat with langsmith",page_icon=":books:")


#@st.cache_resource(show_spinner=False)
@st.cache(show_spinner=False)
def get_response(query):
    #with st.spinner(text="Loading and indexing the Streamlit docs – hang tight! This should take 1-2 minutes."):
        loader = WebBaseLoader("https://python.langchain.com/docs/get_started/quickstart")
        data = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
        all_splits = text_splitter.split_documents(data)
        vectorstore = Chroma.from_documents(documents=all_splits, embedding=VertexAIEmbeddings(model_name="textembedding-gecko@001"))
        # k is the number of chunks to retrieve
        retriever=vectorstore.as_retriever(k=4)
        template = """Create lesson plan or answer the question based on given context:
        {context}

        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)
        model = initialize_llm()

        retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
          | prompt
          | model

        )

        response=retrieval_chain.invoke(query).content

        return response


if "chat_history" not in st.session_state:
    st.session_state.chat_history = [
        AIMessage(content="Hello,How can I help you?"),
    ]

for message in st.session_state.chat_history:
    if isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.write(message.content)
    elif isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.write(message.content)

user_query = st.chat_input("Type your message here...")
if user_query is not None and user_query != "":
    st.session_state.chat_history.append(HumanMessage(content=user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):
        start_time = time.time()
        response = get_response(user_query)
        st.write(stream_data(response))
        end_time = time.time()
        st.write(f"Elapsed time: {end_time-start_time:.2f} seconds")
    st.session_state.chat_history.append(AIMessage(content=response))





Overwriting try.py


In [ ]:
!streamlit run try.py 

2024-04-18 09:05:29.908 INFO    numexpr.utils: NumExpr defaulting to 8 threads.

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.19:8501


  A new version of Streamlit is available.

  See what's new at https://discuss.streamlit.io/c/announcements

  Enter the following command to upgrade:
  $ pip install streamlit --upgrade

2024-04-18 09:05:37.988 Uncaught app exception
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 556, in _run_script
    exec(code, module.__dict__)
  File "/Users/kirtikamerkar/Austincapitaldata/GenAI/notebooks/try.py", line 81, in <module>
    with st.chat_message("AI"):
AttributeError: module 'streamlit' has no attribute 'chat_message'
2024-04-18 09:05:42.775 Uncaught app exception
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/streamlit/runtime/scriptrunner/s